In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import data_getter as dg

pd.get_option("display.max_columns")
pd.set_option("display.max_columns", 999)

In [2]:
player_list_df, team_list_df = dg.get_all_players_n_teams_ids()

/Users/ccuulinay/PycharmProjects/nba_data/nba_analysis/data_getter.py:98: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)


In [3]:
test_df = pd.read_json("data/full_2005-2018.json")
test_df.DOWNLOAD_DATE = pd.to_datetime(test_df.DOWNLOAD_DATE)
test_df.GAME_DATE = pd.to_datetime(test_df.GAME_DATE)
test_df.sort_values(['GAME_DATE'], inplace=True)
test_df.reset_index(inplace=True, drop=True)

In [5]:
test_df.tail(50)

,DOWNLOAD_DATE,GAME_DATE,GAME_TYPE,O_and_U,away_score,away_team_name,company_1_curr_ah_away_odds,company_1_curr_ah_home_odds,company_1_curr_handicap,company_1_curr_ou,company_1_curr_ou_away_odds,company_1_curr_ou_home_odds,company_1_init_ah_away_odds,company_1_init_ah_home_odds,company_1_init_handicap,company_1_init_ou,company_1_init_ou_away_odds,company_1_init_ou_home_odds,company_1_name,company_2_curr_ah_away_odds,company_2_curr_ah_home_odds,company_2_curr_handicap,company_2_curr_ou,company_2_curr_ou_away_odds,company_2_curr_ou_home_odds,company_2_init_ah_away_odds,company_2_init_ah_home_odds,company_2_init_handicap,company_2_init_ou,company_2_init_ou_away_odds,company_2_init_ou_home_odds,company_2_name,company_3_curr_ah_away_odds,company_3_curr_ah_home_odds,company_3_curr_handicap,company_3_curr_ou,company_3_curr_ou_away_odds,company_3_curr_ou_home_odds,company_3_init_ah_away_odds,company_3_init_ah_home_odds,company_3_init_handicap,company_3_init_ou,company_3_init_ou_away_odds,company_3_init_ou_home_odds,company_3_name,company_4_curr_ah_away_odds,company_4_curr_ah_home_odds,company_4_curr_handicap,company_4_curr_ou,company_4_curr_ou_away_odds,company_4_curr_ou_home_odds,company_4_init_ah_away_odds,company_4_init_ah_home_odds,company_4_init_handicap,company_4_init_ou,company_4_init_ou_away_odds,company_4_init_ou_home_odds,company_4_name,company_5_curr_ah_away_odds,company_5_curr_ah_home_odds,company_5_curr_handicap,company_5_curr_ou,company_5_curr_ou_away_odds,company_5_curr_ou_home_odds,company_5_init_ah_away_odds,company_5_init_ah_home_odds,company_5_init_handicap,company_5_init_ou,company_5_init_ou_away_odds,company_5_init_ou_home_odds,company_5_name,handicap,home_score,home_team_name,qiutan_game_id
3379,2018-01-09 00:13:00,2018-01-01 01:00:00,Regular,221,123,Philadelphia 76ers,0.70,1.06,-2.5,220.50,0.90,0.80,0.90,1.00,-2.5,220.50,0.98,0.90,Sbobet,0.95,0.95,-3.5,221.00,0.95,0.95,0.95,0.95,-3,221.00,0.95,0.95,Easybet,0.94,0.94,-3.5,220.50,0.83,0.83,0.98,0.88,-3.5,220.50,0.94,0.90,Crown,0.95,0.86,-3.5,221.00,0.90,0.90,0.90,0.90,-3,221.00,0.90,0.90,Bet365,0.85,0.85,-3.5,221.50,0.85,0.85,0.90,0.90,-3,220.00,0.90,0.90,Vcbet,-3.5,110,Phoenix Suns,290287
3380,2018-01-09 00:13:00,2018-01-02 00:30:00,Regular,219,95,Orlando Magic,0.95,0.81,1.5,218.50,0.95,0.75,1.08,0.82,1,217.00,0.94,0.94,Sbobet,0.95,0.95,1.5,219.00,0.95,0.95,0.95,0.95,2,217.00,0.95,0.95,Easybet,0.81,1.07,2.5,219.50,0.78,0.88,0.87,0.99,2.5,217.50,0.88,0.96,Crown,0.90,0.90,1.5,219.00,0.90,0.90,0.90,0.90,2,217.00,0.90,0.90,Bet365,0.90,0.80,1.5,217.50,1.00,0.75,0.90,0.90,1,217.00,0.90,0.90,Vcbet,1.5,98,Brooklyn Nets,290288
3381,2018-01-09 00:13:00,2018-01-02 00:30:00,Regular,213,127,Milwaukee Bucks,0.80,0.96,6.5,213.50,0.85,0.85,0.94,0.94,6,215.00,0.91,0.95,Sbobet,0.95,0.95,6,213.00,0.95,0.95,0.95,0.95,6,215.00,0.95,0.95,Easybet,0.85,1.03,6.5,213.50,0.80,0.86,0.86,1.00,6.5,214.50,0.92,0.92,Crown,0.86,0.86,5.5,212.50,0.86,0.86,0.90,0.90,6,215.00,0.90,0.90,Bet365,0.90,0.80,5.5,212.50,0.90,0.85,0.90,0.90,6,214.50,0.90,0.90,Vcbet,6,131,Toronto Raptors,290289
3382,2018-01-09 00:13:00,2018-01-02 01:00:00,Regular,201.5,124,Portland Trail Blazers,0.85,0.91,2.5,202.50,0.80,0.90,0.90,1.00,0,206.50,0.94,0.94,Sbobet,0.95,0.95,3,201.50,0.95,0.95,0.95,0.95,-1,206.50,0.95,0.95,Easybet,0.98,0.88,2.5,201.50,0.83,0.83,0.98,0.88,-1.5,206.50,0.92,0.92,Crown,0.86,0.86,2.5,202.50,0.83,0.90,0.90,0.90,-1,206.50,0.90,0.90,Bet365,0.90,0.80,2.5,201.50,0.85,0.85,0.90,0.90,-2.5,207.50,0.90,0.90,Vcbet,3,120,Chicago Bulls,290290
3383,2018-01-09 00:13:00,2018-01-02 01:00:00,Regular,214,96,Los Angeles Lakers,1.01,0.75,9.5,214.50,0.85,0.85,1.03,0.87,10,216.00,0.94,0.94,Sbobet,0.95,0.95,10,214.00,0.95,0.95,0.95,0.95,10.5,216.00,0.95,0.95,Easybet,1.04,0.84,9.5,214.50,0.78,0.88,0.93,0.93,10.5,216.50,0.90,0.94,Crown,0.95,0.80,9.5,213.50,0.86,0.86,0.90,0.90,10.5,216.00,0.90,0.90,Bet365,0.60,1.20,11.5,215.50,0.75,1.00,0.90,0.90,10.5,218.00,0.90,0.90,Vcbet,10,114,Minnesota Timberwolves,290291
3384,2018-01-09

In [6]:
test_run_odds_20180113 = pd.read_json("data/run_odds_20180113.json")
test_run_odds_20180113.DOWNLOAD_DATE = pd.to_datetime(test_run_odds_20180113.DOWNLOAD_DATE)
test_run_odds_20180113.GAME_DATE = pd.to_datetime(test_run_odds_20180113.GAME_DATE)

In [19]:
test_run_odds_20180119 = pd.read_json("data/run_odds_20180119.json")
test_run_odds_20180119.DOWNLOAD_DATE = pd.to_datetime(test_run_odds_20180119.DOWNLOAD_DATE)
test_run_odds_20180119.GAME_DATE = pd.to_datetime(test_run_odds_20180119.GAME_DATE)

In [20]:
test_run_odds_20180119.offer_company_name.unique()

array(['Sbobet ', 'Vcbet', 'Bet365 ', 'Crown ', 'Easybet'], dtype=object)

In [21]:
test_run_odds_20180119.qiutan_game_id.unique()

array([306699, 290407, 290408, 290406])

In [22]:
test_run_odds_20180119[test_run_odds_20180119.qiutan_game_id == 306699].head(50)

,DOWNLOAD_DATE,GAME_DATE,away_curr_score,away_team_name,curr_ah_away_odds,curr_ah_home_odds,curr_handicap,game_run_time,home_curr_score,home_team_name,odds_status,offer_company_name,qiutan_game_id
0,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,1.25,0.56,-10.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
1,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,0.86,0.9,-9.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
2,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,0.77,0.99,-9.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
3,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,0.81,0.95,-9.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
4,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,0.86,0.9,-9.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
5,2018-01-24 13:30:00,2018-01-19,86,Philadelphia 76ers,0.76,1,-9.5,Quarter 4 02:12,76,Boston Celtics,Run,Sbobet,306699
6,2018-01-24 13:30:00,2018-01-19,84,Philadelphia 76ers,1.25,0.56,-12.5,Quarter 4 03:31,71,Boston Celtics,Run,Sbobet,306699
7,2018-01-24 13:30:00,2018-01-19,84,Philadelphia 76ers,1.11,0.66,-12.5,Quarter 4 03:31,71,Boston Celtics,Run,Sbobet,306699
8,2018-01-24 13:30:00,2018-01-19,84,Philadelphia 76ers,1.19,0.6,-12.5,Quarter 4 03:31,71,Boston Celtics,Run,Sbobet,306699
9,2018-01-24 13:30:00,2018-01-19,84,Philadelphia 76ers,0.86,0.9,-12.5,Quarter 4 03:44,69,Boston Celtics,Run,Sbobet,306699


In [23]:
pp_0021700663_df = pd.read_csv("data/pp/pp_0021700663.csv")

In [24]:
pp_0021700663_df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,21700663,2,12,0,1,7:03 PM,12:00,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,21700663,4,10,0,1,7:03 PM,12:00,Jump Ball Horford vs. Embiid: Tip to Simmons,NaN,NaN,NaN,NaN,4,201143,Al Horford,1.610613e+09,Boston,Celtics,BOS,5,203954,Joel Embiid,1.610613e+09,Philadelphia,76ers,PHI,5,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI
2,21700663,7,5,45,1,7:03 PM,11:46,NaN,NaN,Bayless Out of Bounds - Bad Pass Turnover Turn...,NaN,NaN,5,201573,Jerryd Bayless,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN
3,21700663,8,2,1,1,7:04 PM,11:32,MISS Brown 3PT Jump Shot,NaN,NaN,NaN,NaN,4,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
4,21700663,9,4,0,1,7:04 PM,11:31,NaN,NaN,Saric REBOUND (Off:0 Def:1),NaN,NaN,5,203967,Dario Saric,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
5,21700663,10,2,98,1,7:04 PM,11:18,NaN,NaN,MISS Saric 1' Cutting Layup Shot,NaN,NaN,5,203967,Dario Saric,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
6,21700663,11,4,0,1,7:04 PM,11:17,Baynes REBOUND (Off:0 Def:1),NaN,NaN,NaN,NaN,4,203382,Aron Baynes,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
7,21700663,12,1,1,1,7:04 PM,11:07,Horford 15' Jump Shot (2 PTS),NaN,NaN,0 - 2,2,4,201143,Al Horford,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
8,21700663,13,5,45,1,7:05 PM,10:54,NaN,NaN,Simmons Out of Bounds - Bad Pass Turnover Turn...,NaN,NaN,5,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN
9,21700663,14,2,79,1,7:05 PM,10:41,MISS Brown 18' Pullup Jump Shot,NaN,NaN,NaN,NaN,4,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
